# Comparativa de los tweets sobre la serie "Big Bang Theory" y "Silicon Valley" 

En este trabajo voy a comparar cual de las dos series está gustando más, usanto las técnicas de análisis de sentimientos en base a los tweets extraidos de tweeter. 

In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import datetime
import time
import csv
import json
import nltk
import re
from pprint import pprint

from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
import re

import random

from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [67]:
consumer_key = '' # aqui van is claves de desarrollador de tweeter
consumer_secret = ''
access_token = ''
access_token_secret = ''

### Análisis de la serie "Big Bang Theory"

In [3]:
#con esta función vamos a extraer los tweets sobre la serie "Big Bang Theory"
def searchBigBangTheory(hashtag, Max_Tweets, Recdata):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='en').items(Max_Tweets)
    
    current_working_dir = "./"
    log_tweets = current_working_dir  + 'JSONtweets.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
#guardamos tweets en un data frame
    tweets = pd.DataFrame()
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)

#guadramos tweets en csv

    tweets.to_csv('./CVStweetsBigBangTheory.csv', sep=';', header=False)

In [4]:
Max_Tweets = 1000 # voy a extraer mil últimos tweets
tweets_data = []
hashtag = "BigBangTheory"

searchBigBangTheory(hashtag, Max_Tweets, tweets_data)

In [5]:
tweet_list = [] # creando una lista de tweets
with open('./JSONtweets.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

In [11]:
tweet_list[:10]

['@bigbangtheory @simonhelberg happy birthday',
 'RT @bigbangtheory: RETWEET to wish @simonhelberg a happy birthday! Here are 14 facts you needs to know about the #BigBangTheory star: https\xe2\x80\xa6',
 '5 days!! 5 fucking days!! FIVE DAYS!! #BigBangTheory 10X11 #Shamy sheldon and amy https://t.co/bwb97sneMK',
 'RT @bigbangtheory: RETWEET to wish @simonhelberg a happy birthday! Here are 14 facts you needs to know about the #BigBangTheory star: https\xe2\x80\xa6',
 'RT @TeamCoco: "They treat #BigBangTheory like a reality show." - @MelissaRauch\'s parents thought she was ACTUALLY pregnant https://t.co/up7\xe2\x80\xa6',
 'RT @CBSTweet: Double tap to wish #simonhelberg a happy birthday! #BigBangTheory https://t.co/2gQmcyIeQp',
 'RT @CBSTweet: 2016 was full of surprises! Here are our favorite CBS surprises from 2016! #NCIS #BigBangTheory #LifeInPieces #BlueBloods #Su\xe2\x80\xa6',
 'Nothing more depressing than coming to the end of @bigbangtheory for the 10000000876500th time.',
 'RT @Tea

In [12]:
#Con esta función se van a formatear los tweets
def processTweet(tweet):
    tweet = tweet.lower()
    # www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #@username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

stopWords = []

#quitando caracteres repetidos
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [13]:
# Dividimos los tweets en palabras
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [14]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

In [15]:

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def read_corpus(fname, t_type):
    tweets = []
    f = open(fname, 'r')
    line = f.readline()
    while line != '':
        tweets.append([line, t_type])
        line = f.readline()
    f.close()
    return tweets

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

def classify_tweet(tweet):
    return classifier.classify(extract_features(nltk.word_tokenize(tweet)))

In [16]:
# palabras positivas y negativas que voy a usar para entrenar, podría entrenar con un corpus de tweets etiquetados pero tengo lista de palabras
positivos = read_corpus('./positiveMcDonalds.csv', 'POS')
no_positivos = read_corpus('./negativeMcDonalds.csv', 'NO_POS')

In [17]:
# construyo el set de entrenamiento con las palabras positivas y negativas del corpus
tweets = []
for (words, sentiment) in positivos + no_positivos:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

word_features = get_word_features(get_words_in_tweets(tweets))

# get the training set and train the Naive Bayes Classifier
training_set = nltk.classify.util.apply_features(extract_features, tweets)
classifier = NaiveBayesClassifier.train(training_set)

In [18]:
NBClassifier = nltk.NaiveBayesClassifier.train(training_set) # entreno con el algoritmo Naive Bayes

# A continuación clasifico los tweets sobre la serie "Big Bang Theory".
for tweet in tweet_list[:10]:
    processedTestTweet = processTweet(tweet)
    print tweet + '  ----->  ' + NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) + '\n'

@bigbangtheory @simonhelberg happy birthday  ----->  POS

RT @bigbangtheory: RETWEET to wish @simonhelberg a happy birthday! Here are 14 facts you needs to know about the #BigBangTheory star: https…  ----->  POS

5 days!! 5 fucking days!! FIVE DAYS!! #BigBangTheory 10X11 #Shamy sheldon and amy https://t.co/bwb97sneMK  ----->  NO_POS

RT @bigbangtheory: RETWEET to wish @simonhelberg a happy birthday! Here are 14 facts you needs to know about the #BigBangTheory star: https…  ----->  POS

RT @TeamCoco: "They treat #BigBangTheory like a reality show." - @MelissaRauch's parents thought she was ACTUALLY pregnant https://t.co/up7…  ----->  POS

RT @CBSTweet: Double tap to wish #simonhelberg a happy birthday! #BigBangTheory https://t.co/2gQmcyIeQp  ----->  POS

RT @CBSTweet: 2016 was full of surprises! Here are our favorite CBS surprises from 2016! #NCIS #BigBangTheory #LifeInPieces #BlueBloods #Su…  ----->  POS

Nothing more depressing than coming to the end of @bigbangtheory for the 10000000

In [116]:
# Voy a contar los tweets positivos para poder comparar con la otra serie.
count = 0
for tweet in tweet_list:
    
    processedTestTweet = processTweet(tweet)
    if NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) == 'POS': count=count+1
    print NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))  + '\n'
    print count

NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
NO_POS

0
POS

1
NO_POS

1
POS

2
NO_POS

2
NO_POS

2
NO_POS

2
POS

3
POS

4
NO_POS

4
NO_POS

4
NO_POS

4
POS

5
POS

6
POS

7
POS

8
POS

9
NO_POS

9
POS

10
NO_POS

10
NO_POS

10
NO_POS

10
NO_POS

10
POS

11
NO_POS

11
NO_POS

11
NO_POS

11
POS

12
POS

13
POS

14
NO_POS

14
POS

15
POS

16
NO_POS

16
POS

17
POS

18
NO_POS

18
NO_POS

18
NO_POS

18
POS

19
NO_POS

19
POS

20
POS

21
POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
NO_POS

22
POS

23
NO_POS

23
NO_POS

23
NO_POS

23
NO_POS

23
POS

24
NO_POS

24
NO_POS

24
NO_POS

24
POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
NO_POS

25
N

202 tweets sobre la serie "Big Bang Theory" han sido positivos.

### Análisis de la serie "Silicon Valley"

In [24]:
#con esta función vamos a extraer los tweets sobre la serie "Silicon Valley"
def searchSiliconValley(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='en').items(Max_Tweets)
    
    current_working_dir = "./"
    log_tweets = current_working_dir  + 'JSONtweets.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
#guardamos tweets en un data frame
    tweets = pd.DataFrame()
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
#guadramos tweets en csv

    tweets.to_csv('./CVStweetsSiliconValley.csv', sep=';', header=False)

In [25]:
Max_Tweets = 1000 # voy a extraer mil últimos tweets
tweets_data = []
hashtag = "SiliconValleyHBO"

searchSiliconValley(hashtag, Max_Tweets, tweets_data)

In [26]:
tweet_list = []  # creando una lista de tweets
with open('./JSONtweets.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

In [27]:
tweet_list[:10]

['RT @SiliconHBO: Congratulations to the writers of #SiliconValleyHBO on the #WGA nomination for best writing for a comedy series. https://t.\xe2\x80\xa6',
 '#SiliconValleyHBO needs to incorporate all these fights the actors are having with Trumpkins',
 "RT @SiliconVallism: Jared: We're not here to tell you what to do with your rats, we're here to get your rats, STAT. #SiliconValley #Silicon\xe2\x80\xa6",
 "Jared: We're not here to tell you what to do with your rats, we're here to get your rats, STAT. #SiliconValley #SiliconHBO #SiliconValleyHBO",
 "Dinesh: Did you see that? She gave me her hat.\nGilfoyle: Pretend you've seen a woman before. #SiliconValley #SiliconHBO #SiliconValleyHBO",
 'RT @SiliconHBO: Congratulations to the writers of #SiliconValleyHBO on the #WGA nomination for best writing for a comedy series. https://t.\xe2\x80\xa6',
 'Learn to understand HBO\xe2\x80\x99s hilarious show Silicon Valley one Learning Library post at a time. #SiliconValleyHBO https://t.co/N4o58zwANo

In [28]:
# Clasificación de los tweets de la serie "Silicon Valley"
for tweet in tweet_list[:10]:
    processedTestTweet = processTweet(tweet)
    print tweet + '  ----->  ' + NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) + '\n'

RT @SiliconHBO: Congratulations to the writers of #SiliconValleyHBO on the #WGA nomination for best writing for a comedy series. https://t.…  ----->  POS

#SiliconValleyHBO needs to incorporate all these fights the actors are having with Trumpkins  ----->  NO_POS

RT @SiliconVallism: Jared: We're not here to tell you what to do with your rats, we're here to get your rats, STAT. #SiliconValley #Silicon…  ----->  NO_POS

Jared: We're not here to tell you what to do with your rats, we're here to get your rats, STAT. #SiliconValley #SiliconHBO #SiliconValleyHBO  ----->  NO_POS

Dinesh: Did you see that? She gave me her hat.
Gilfoyle: Pretend you've seen a woman before. #SiliconValley #SiliconHBO #SiliconValleyHBO  ----->  NO_POS

RT @SiliconHBO: Congratulations to the writers of #SiliconValleyHBO on the #WGA nomination for best writing for a comedy series. https://t.…  ----->  POS

Learn to understand HBO’s hilarious show Silicon Valley one Learning Library post at a time. #SiliconValleyHB

In [29]:
# Voy a contar los tweets positivos de la serie "Silicon Valley"
count = 0
for tweet in tweet_list:
    
    processedTestTweet = processTweet(tweet)
    if NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) == 'POS': count=count+1
    print NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))  + '\n'
    print count

POS

1
NO_POS

1
NO_POS

1
NO_POS

1
NO_POS

1
POS

2
NO_POS

2
NO_POS

2
NO_POS

2
NO_POS

2
POS

3
POS

4
NO_POS

4
NO_POS

4
NO_POS

4
NO_POS

4
NO_POS

4
NO_POS

4
POS

5
POS

6
NO_POS

6
NO_POS

6
POS

7
POS

8
NO_POS

8
NO_POS

8
POS

9
NO_POS

9
NO_POS

9
POS

10
POS

11
POS

12
POS

13
POS

14
POS

15
POS

16
POS

17
POS

18
POS

19
POS

20
POS

21
POS

22
POS

23
POS

24
POS

25
POS

26
POS

27
POS

28
POS

29
POS

30
POS

31
POS

32
POS

33
POS

34
POS

35
POS

36
POS

37
POS

38
POS

39
POS

40
POS

41
POS

42
POS

43
POS

44
POS

45
POS

46
POS

47
POS

48
POS

49
POS

50
POS

51
NO_POS

51
POS

52
POS

53
POS

54
POS

55
POS

56
POS

57
POS

58
POS

59
POS

60
POS

61
POS

62
POS

63
POS

64
POS

65
POS

66
POS

67
POS

68
POS

69
POS

70
POS

71
POS

72
POS

73
POS

74
POS

75
POS

76
POS

77
POS

78
POS

79
POS

80
POS

81
POS

82
POS

83
POS

84
POS

85
POS

86
POS

87
POS

88
POS

89
POS

90
POS

91
NO_POS

91
POS

92
POS

93
POS

94
POS

95
POS

96
POS

97
POS

98
POS

127 tweets sobre la serie "Silicon Valley" han sido positivos.